# Run valuation

#### Step 1: Create instruments

```sql
-- ============================================================
-- Description:
-- In this file, we load instrument data from the Excel sheet
-- ============================================================

-- Load worksheet from Excel file which contains instrument data

@instruments_data = use Drive.Excel
--file=/luminesce-examples/simplified_valuation_data.xlsx
--worksheet=instruments
enduse;

-- Run instruments transformation

@instruments_for_upload = select
name as DisplayName,
figi as Figi,
currency as DomCcy,
'RunValuation' as Scope
from @instruments_data;

-- Upload the transformed data into LUSID

select *
from Lusid.Instrument.Equity.Writer
where ToWrite = @instruments_for_upload;

```

#### Step 2: Create portfolio

```sql
-- ============================================================
-- Description:
-- In this file, we load portfolio data from the Excel sheet
-- ============================================================

-- Defining scope and code variables

@@portfolioScope = select 'ibor';
@@portfolioCode1 = select 'uk-equity';

-- Define the portfolio data

@create_portfolio = select 'Transaction' as PortfolioType,
@@portfolioScope as PortfolioScope,
@@portfolioCode1 as PortfolioCode,
'RunValuation' as InstrumentScopes,
@@portfolioCode1 as DisplayName,
'' as Description,
#2000-01-01# as Created,
'' as SubHoldingKeys,
'GBP' as BaseCurrency;

-- Upload the portfolio into LUSID

select *
from Lusid.Portfolio.Writer
where ToWrite = @create_portfolio;

```

#### Step 3: Upload quotes

```sql
-- ========================================================
-- Description:
-- In this file, we load quote data from the Excel sheet
-- ========================================================

-- Load quotes from an Excel worksheet

@quotes_data = use Drive.Excel
--file=/luminesce-examples/simplified_valuation_data.xlsx
--worksheet=prices
enduse;

-- Transform quote data

@quotes_for_upload = select
'Figi' as InstrumentIdType,
figi as Instrumentid,
'ibor' as QuoteScope,
'Price' as QuoteType,
'Lusid' as Provider,
'Mid' as Field,
price_date as QuoteEffectiveAt,
close_price as Value,
'GBP' as Unit
from @quotes_data;

-- Upload quotes into LUSID

select * from Lusid.Instrument.Quote.Writer
where ToWrite = @quotes_for_upload;

```

#### Step 4: Upload transactions

```sql
-- ============================================================
-- Description:
-- In this file, we load transaction data from the Excel sheet
-- ============================================================

-- Load transaction from Excel file on Drive

@transactions_data = use Drive.Excel
--file=/luminesce-examples/simplified_valuation_data.xlsx
--worksheet=transactions
enduse;

-- Transform transactions data

@txns =
select
instrument_id as Figi,
portfolio_code as PortfolioCode,
'ibor' as PortfolioScope,
currency as TradeCurrency,
currency as SettlementCurrency,
txn_trade_date as TransactionDate,
txn_settle_date  as SettlementDate,
txn_price as TradePrice,
txn_id as TxnId,
txn_type as Type,
txn_consideration as TotalConsideration,
txn_units as Units,
'Upsert' as WriteAction,
'RunValuation' as InstrumentScope
from @transactions_data;

-- load transactions into LUSID

select * from Lusid.Portfolio.Txn.Writer
where ToWrite = @txns;

```

#### Step 5: Run simple valuation

```sql
-- ============================================================
-- Description:
-- In this file, we run a valuation using the data which was
-- previously loaded. You also need a recipe which is setup by
-- the _data/setup.py file.
-- ============================================================

-- Select some metrics to be returned by the valuation engine

@metrics = values
('Instrument/default/Name', 'Value'),
('Instrument/default/Figi', 'Value'),
('Valuation/PV/Ccy', 'Value'),
('Valuation/PV', 'Value'),
('Valuation/PvInPortfolioCcy', 'Value'),
('Valuation/PvInPortfolioCcy', 'Proportion');

@measures = select column1 as MeasureName, column2 as Operation from @metrics;

-- Run the valuation for a given recipe, portfolio and date

@vals = select *
from lusid.portfolio.valuation
where PortfolioCode = 'uk-equity'
   and PortfolioScope = 'ibor'
   and Recipe = 'ibor/market-value'
   and MeasuresToReturn = @measures
   and EffectiveAt = '2020-08-24T09:00:00.000Z';

-- Pivot the values into a traditional report format

@vals_formatted =
use Tools.Pivot with @vals
--key=MeasureName
--aggregateColumns=MeasureDecimalValue,MeasureStringValue
enduse;

select
PortfolioCode as 'Portfolio',
ValuationDate as 'Val Date',
[Instrument/default/Name_MeasureStringValue] as 'InstrumentName',
[Instrument/default/Figi_MeasureStringValue] as 'Figi',
[Valuation/PV/Ccy_MeasureStringValue] as 'Currency',
[Valuation/PV_MeasureDecimalValue] as 'MarketValue',
[Valuation/PvInPortfolioCcy_MeasureDecimalValue] as 'MarketValuePortfolioCcy',
[Proportion(Valuation/PvInPortfolioCcy)_MeasureDecimalValue] as  'MarketValuePct'
from @vals_formatted;

```

#### Step 6: Upload external valuations

```sql
-- ============================================================================
-- Description:
-- In this file, we upsert external valuations into the Structured Result Store
-- External valuations are loaded from a file located in LUSID Drive
-- ============================================================================

-- Load a CSV file of valuations from LUSID Drive

@valuations = use Drive.Csv
--file=/luminesce-examples/external_valuations.csv
enduse;

@valuations_with_luid =
select val.PortfolioScope,
val.PortfolioCode,
val.ValuationDate,
eq.LusidInstrumentId as InstrumentId,
val.Currency,
val.[External-MarketValue]
from @valuations val
inner join lusid.instrument.equity eq
where val.Figi = eq.Figi
and Scope = 'RunValuation';


-- Create SRS Document

@header = values ('PortfolioScope','PortfolioCode','ValuationDate','InstrumentId','Currency','External-MarketValue');

@header_valuations = 
select 
column1 as PortfolioScope,
column2 as PortfolioCode,
column3 as ValuationDate,
column4 as InstrumentId,
column5 as Currency,
column6 as [External-MarketValue] from @header
union
select * from @valuations_with_luid;


@y = 
select 
PortfolioScope   
|| ','
|| PortfolioCode
|| ','
|| ValuationDate 
|| ','
|| InstrumentId
|| ','
|| Currency
|| ','
|| [External-MarketValue]
 as a_row
from @header_valuations
order by PortfolioScope desc;

@@doc = select group_concat(a_row, '
') x from @y;

-- Upsert valuations Document

@data_to_upsert = 
select 
'ibor' as Scope,
'MarketValuation' as Code,
'UnitResult/Holding' as ResultType,
'Client' as Source,
'Market valuations' as Name,
'market-srs-valuation-map' as DataMapCode,
'0.1.1' as DataMapVersion,
'0.1.1' as Version,
'Csv' as [Format],
@@doc as Document,
'2020-08-24T09:00:00.000Z' as EffectiveAt,
'Upsert' as WriteAction;

select * from Lusid.UnitResult.StructuredResult.Writer where ToWrite = @data_to_upsert;
```

#### Step 7: Run valuation with srs

```sql
-- ============================================================
-- Description:
-- In this file, we run a valuation using the data which was
-- previously loaded, including valuations from SRS 
-- You also need a recipe which is setup by the _data/setup.py file.
-- ============================================================

-- Select some metrics to be returned by the valuation engine

@measure =
select 'Instrument/default/Name' as MeasureName, 'Value' as Operation union
select 'Valuation/PV/Ccy' as MeasureName, 'Value' as Operation union
select 'Valuation/PV/Amount' as MeasureName, 'Sum' as Operation union
select 'UnitResult/External-MarketValue' as MeasureName, 'Sum' as Operation; -- retrieved from SRS

-- Run the valuation for a given recipe, portfolio and date

@vals = 
select *
from lusid.portfolio.valuation
where PortfolioCode = 'uk-equity'
   and PortfolioScope = 'ibor'
   and Recipe = 'ibor/market-value'
   and MeasuresToReturn = @measure
   and EffectiveAt = '2020-08-24T09:00:00.000Z';

-- Pivot the values into a traditional report format

@vals_formatted =
use Tools.Pivot with @vals
--key=MeasureName
--aggregateColumns=MeasureDecimalValue,MeasureStringValue
enduse;

select
PortfolioCode as 'Portfolio',
ValuationDate as 'Val Date',
[Instrument/default/Name_MeasureStringValue] as 'InstrumentName',
[Valuation/PV/Ccy_MeasureStringValue] as 'Currency',
[Sum(Valuation/PV/Amount)_MeasureDecimalValue] as 'MarketValue',
[Sum(UnitResult/External-MarketValue)_MeasureDecimalValue] as 'External-MarketValue'
from @vals_formatted;

```

#### README.md

```sql
# Running valuations

In this section we use Luminesce to run a valuation on an equity portfolio. Specifically we cover the following:

1. Create the equity instruments (these are loaded from a worksheet in an Excel file which is stored in LUSID Drive)
2. Create a portfolio.
3. Upload quotes (also loaded from the same Excel file on Drive)
4. Upload transactions (also loaded from the same Excel file on Drive)
5. Run a valuation.
6. Upload external market value into Structured Result Store (loaded from csv file on Drive)
7. Run a valuation including both LUSID and External Market Value

See this [support page](https://support.lusid.com/knowledgebase/article/KA-01678/en-us) for more details on running valuations via the Luminesce Webtool.

## Source data

* We load prices, transaction and quotes from an Excel file on LUSID drive
* We load external valuations from a csv file on LUSID drive
* We use [Drive.Excel](https://support.lusid.com/knowledgebase/article/KA-01682/en-us) and [Drive.Csv](https://support.lusid.com/knowledgebase/article/KA-01680/) providers to load the data into Luminesce

## Implementation details

* The recipe used in the valuation is created in the `setup.py` file
* The SRS data map is also created in `setup.py` file
* By default, the valuation provider returns a row per instrument/metric. We then use the `pivot` provider to present the data in a more traditional tabular format.

```